### **Pipeline Completo de Machine Learning para Previsão de Churn (Versão Google Colab)**

Este notebook implementa um pipeline completo para prever a evasão de clientes (churn) de uma empresa de telecomunicações. As etapas incluem carga de dados, pré-processamento, balanceamento de classes, remoção de multicollinearidade e treinamento de modelos de Machine Learning.

#### **Célula 1: Instalação de Bibliotecas (Opcional)**
*O Google Colab geralmente já possui estas bibliotecas. Execute esta célula apenas se encontrar um erro de importação.*

In [ ]:
# !pip install pandas numpy matplotlib seaborn scikit-learn imbalanced-learn statsmodels

#### **Célula 2: Importação das Bibliotecas**
*Nesta célula, importamos todas as ferramentas necessárias para o projeto.*

In [ ]:
# Importar bibliotecas básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io

# Importar ferramentas do Scikit-learn e Imbalanced-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE

# Ferramenta para análise de multicollinearidade
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Ferramenta para upload de arquivo no Google Colab
from google.colab import files

# Configuração de visualização para pandas e matplotlib
pd.options.display.float_format = '{:.2f}'.format
sns.set_style("whitegrid")

#### **Célula 3: Carga e Limpeza Inicial dos Dados**
* **Ação Necessária:** Execute esta célula. Um botão "Escolher arquivos" aparecerá. Clique nele e selecione o arquivo `df_limpo.csv` do seu computador.
* **O que acontece:** O código fará o upload do seu arquivo para o ambiente do Colab e, em seguida, o carregará em um DataFrame. As etapas de limpeza e renomeação de colunas serão aplicadas na sequência.

In [ ]:
# 1. Faz o upload do arquivo para o ambiente do Colab
print("Por favor, selecione o arquivo 'df_limpo.csv'")
uploaded = files.upload()

# 2. Carrega o arquivo em um DataFrame do pandas
# O io.BytesIO é usado para ler o arquivo que está em memória
file_name = list(uploaded.keys())[0]
df = pd.read_csv(io.BytesIO(uploaded[file_name]))

print(f"\nArquivo '{file_name}' carregado com sucesso!")

# --- 3. Limpeza e Pré-processamento Inicial ---

# Padronizar nomes das colunas (substituir '.' por '_')
df.columns = df.columns.str.replace('.', '_', regex=False)

# Tratar os valores nulos com 0
df['total_day'] = df['total_day'].fillna(0)
df['account_charges_total'] = df['account_charges_total'].fillna(0)

# Remover a coluna de ID, que é irrelevante para o modelo
df = df.drop(columns=['customerID'])

print("\n--- Visualização dos dados após limpeza inicial ---")
df.head()

#### **Célula 4: Codificação de Variáveis Categóricas (Encoding)**
*Aqui, transformamos colunas de texto (categóricas) em um formato numérico que o modelo possa entender, usando a técnica de One-Hot Encoding.*

In [ ]:
# Identificar colunas categóricas (tipo 'object')
colunas_categoricas = df.select_dtypes(include=['object']).columns

# Aplicar One-Hot Encoding
df_codificado = pd.get_dummies(df, columns=colunas_categoricas, drop_first=True)

# Renomear a coluna alvo para maior clareza
df_codificado = df_codificado.rename(columns={'churn_Yes': 'churn_sim'})

print("--- DataFrame após a codificação das variáveis ---")
df_codificado.info()

#### **Célula 5: Divisão dos Dados e Balanceamento com SMOTE**
*Dividimos os dados em conjuntos de treino e teste. Em seguida, usamos a técnica SMOTE para balancear as classes no conjunto de treino, evitando que o modelo fique viciado na classe majoritária.*

In [ ]:
# 1. Separar as features (x) da variável alvo (y)
x = df_codificado.drop(columns=['churn_sim'])
y = df_codificado['churn_sim']

# 2. Dividir os dados em conjuntos de treino e teste
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.2, stratify=y, random_state=42)

# 3. Aplicar SMOTE apenas no conjunto de treino
smote = SMOTE(random_state=42)
x_treino_balanceado, y_treino_balanceado = smote.fit_resample(x_treino, y_treino)

print("\n--- Distribuição das classes ---")
print("Antes do SMOTE (treino):")
print(y_treino.value_counts())
print("\nDepois do SMOTE (treino):")
print(y_treino_balanceado.value_counts())

#### **Célula 6: Padronização das Features Numéricas (Scaling)**
*Colocamos todas as features numéricas na mesma escala. Isso é importante para modelos como a Regressão Logística e melhora a performance geral.*

In [ ]:
# Identificar as colunas numéricas para padronização
colunas_numericas = x_treino_balanceado.select_dtypes(include=np.number).columns.tolist()

# Criar e treinar o scaler COM OS DADOS DE TREINO
scaler = StandardScaler()
x_treino_padronizado = scaler.fit_transform(x_treino_balanceado)

# Apenas transformar os DADOS DE TESTE com o scaler já treinado
x_teste_padronizado = scaler.transform(x_teste)

# Recriar os DataFrames com os dados padronizados, mantendo os nomes das colunas
x_treino_padronizado = pd.DataFrame(x_treino_padronizado, columns=x_treino_balanceado.columns)
x_teste_padronizado = pd.DataFrame(x_teste_padronizado, columns=x_teste.columns)

print("\n--- Amostra dos dados de treino após padronização ---")
x_treino_padronizado.head()

#### **Célula 7: Análise e Remoção de Multicollinearidade com VIF**
*Verificamos a redundância entre as features. Features com VIF (Fator de Inflação da Variância) muito alto são removidas para aumentar a estabilidade e a interpretabilidade do modelo.*

In [ ]:
# Calcular o VIF para cada feature
vif_data = pd.DataFrame()
vif_data["feature"] = x_treino_padronizado.columns
vif_data["VIF"] = [variance_inflation_factor(x_treino_padronizado.values, i) for i in range(len(x_treino_padronizado.columns))]

print("--- Fator de Inflação da Variância (VIF) antes da remoção ---")
print(vif_data.sort_values(by='VIF', ascending=False))

# Definir colunas a serem removidas com base no VIF (infinito ou > 10)
colunas_para_remover_vif = [
    'internet_online_security_No internet service',
    'internet_online_backup_No internet service',
    'internet_device_protection_No internet service',
    'internet_tech_support_No internet service',
    'internet_streaming_tv_No internet service',
    'internet_streaming_movies_No internet service',
    'account_charges_total',
    'total_day'
]

# Criar os conjuntos de dados finais para treinamento, removendo as colunas
x_treino_final = x_treino_padronizado.drop(columns=colunas_para_remover_vif, errors='ignore')
x_teste_final = x_teste_padronizado.drop(columns=colunas_para_remover_vif, errors='ignore')

print(f"\nNúmero de features originais: {x_treino_padronizado.shape[1]}")
print(f"Número de features finais após remoção por VIF: {x_treino_final.shape[1]}")

#### **Célula 8: Treinamento e Avaliação dos Modelos**
*Finalmente, treinamos e avaliamos dois modelos de classificação com nosso conjunto de dados final.*

In [ ]:
# --- MODELO 1: REGRESSÃO LOGÍSTICA ---
print("\n---------------------------------------")
print("   Modelo: Regressão Logística")
print("---------------------------------------")
modelo_lr = LogisticRegression(random_state=42, max_iter=1000)
modelo_lr.fit(x_treino_final, y_treino_balanceado)
predicoes_lr = modelo_lr.predict(x_teste_final)

acuracia_lr = accuracy_score(y_teste, predicoes_lr)
relatorio_lr = classification_report(y_teste, predicoes_lr, target_names=['Não Churn', 'Churn'])

print(f"Acurácia: {acuracia_lr:.4f}")
print("Relatório de Classificação:\n", relatorio_lr)


# --- MODELO 2: RANDOM FOREST ---
print("\n---------------------------------------")
print("   Modelo: Random Forest")
print("---------------------------------------")
modelo_rf = RandomForestClassifier(random_state=42, n_estimators=100) # n_estimators é um bom parâmetro para ajustar
modelo_rf.fit(x_treino_final, y_treino_balanceado)
predicoes_rf = modelo_rf.predict(x_teste_final)

acuracia_rf = accuracy_score(y_teste, predicoes_rf)
relatorio_rf = classification_report(y_teste, predicoes_rf, target_names=['Não Churn', 'Churn'])

print(f"Acurácia: {acuracia_rf:.4f}")
print("Relatório de Classificação:\n", relatorio_rf)

#### **Célula 9: Análise de Importância das Features do Melhor Modelo (Random Forest)**
*Visualizamos quais fatores o modelo Random Forest considerou mais importantes para prever o churn.*

In [ ]:
# Criar um DataFrame com a importância de cada feature
importancia_features = pd.DataFrame({
    'Feature': x_treino_final.columns,
    'Importancia': modelo_rf.feature_importances_
}).sort_values(by='Importancia', ascending=False)

print("\n--- Top 10 Features Mais Importantes (Random Forest) ---")
print(importancia_features.head(10))

# Gerar o gráfico de barras
plt.figure(figsize=(10, 8))
sns.barplot(x='Importancia', y='Feature', data=importancia_features, palette='viridis')
plt.title('Importância das Features no Modelo Random Forest Final', fontsize=16)
plt.xlabel('Nível de Importância', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.tight_layout()
plt.show()

#### **Célula 10: Conclusão Estratégica**

### Conclusão Estratégica e Próximos Passos

**1. Modelo Escolhido:** O modelo **Random Forest** apresentou o melhor equilíbrio entre as métricas, especialmente no **Recall** para a classe "Churn". Isso significa que ele é mais eficaz em identificar os clientes que realmente têm a intenção de sair, o que é crucial para ações de retenção. O modelo é robusto e confiável, pois foi treinado em dados balanceados e livres de multicollinearidade.

**2. Principais Fatores de Evasão:**
A análise de importância das features revelou os seguintes fatores como os mais críticos:
- **`customer_tenure` (Tempo de Contrato):** Clientes mais novos são mais propensos a sair.
- **`account_charges_monthly` (Cobrança Mensal):** O valor da fatura é um fator decisivo.
- **`account_contract_...` (Tipo de Contrato):** Planos mensais representam o maior risco.
- **`account_payment_method_Electronic check` (Pagamento com Cheque Eletrônico):** Forte indicador de risco.
- **`internet_internet_service_Fiber optic` (Internet Fibra Óptica):** Surpreendentemente, um fator de risco que merece investigação.

**3. Recomendações Acionáveis:**
- **Fidelização:** Criar campanhas agressivas para migrar clientes de planos mensais para anuais, oferecendo descontos ou benefícios.
- **Onboarding de Clientes:** Desenvolver um programa de acompanhamento nos primeiros 3-6 meses para novos clientes, garantindo uma boa experiência inicial.
- **Análise de Portfólio:** Realizar uma pesquisa de satisfação com clientes de Fibra Óptica para entender as razões da alta evasão (preço, qualidade, concorrência).
- **Modernização de Pagamento:** Incentivar a migração do cheque eletrônico para débito automático com um pequeno bônus na fatura.

**Próximos Passos:** O modelo está pronto para ser implementado em um ambiente de produção, onde pode ser integrado ao CRM da empresa para gerar uma pontuação de risco de churn para cada cliente em tempo real, permitindo que as equipes de retenção atuem de forma proativa.